In [357]:
import pickle
import ml.structures
import ml.dataset.corpus as corpus
import numpy as np

import pickle
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from keras.models import load_model

from keras.layers import LSTM
from keras.layers import Embedding
from keras.optimizers import RMSprop


from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras.engine.topology import Input

from sklearn.metrics import accuracy_score
from keras.layers import concatenate, Merge
from keras.layers.merge import Concatenate

import keras
from keras.utils import to_categorical

from matplotlib import pyplot as plt
from IPython.display import clear_output

In [358]:
def make_16(chord):
    if (chord.duration > 0):
        c = 16 / (128 / chord.duration)
    else:
        return 0, [0]
    array = np.array([note.number for note in chord.notes])
    if len(array) == 0:
        array = [0]
    if c >= 1:
        return 1, np.tile(array, (int(c), 1))
    else:
        return 0, [0]

In [359]:
def cut_song(song, notes, chords):
    chords_track = []
    notes_track = np.array([])
    flag = 0
    for note in song.tracks[0].chords:
        if (make_16(note)[0] == 0):
            flag = 1
        notes_track = np.append(notes_track, make_16(note)[1])
    for chord in song.tracks[1].chords:
        if (make_16(chord)[0] == 0):
            flag = 1
        chords_track.append(make_16(chord)[1])
    if flag == 1:
        return 0, notes, chords
    return 1, notes_track, chords_track

In [360]:
def norm_array(X, size):
    for j in range(len(X)):
        X[j] = np.unique(np.array(X[j]))[:size]
        array = np.zeros(size-len(X[j])) - 1
        X[j] = np.append(X[j], array)
    X = np.array(X)
    return X

In [361]:
def create_X_Y(chords):
    real_chords = []
    y = []
    for i, subchord in enumerate(chords):
        if i != len(chords) - 1:
            for ssubchord in subchord:
                real_chords.append(ssubchord)
            size = len(subchord)
            for j in range(size):
                y.append(chords[i+1][-1])
    return real_chords, y

In [362]:
def notes_to_np(notes):
    for i in range(len(notes)):
        notes[i] = np.array(notes[i])
    notes = np.array(notes)

In [363]:
def create_unique_chords(X, unique_chords):
    for x in X:
        unique_chords = np.append(unique_chords, np.array(x))
    return unique_chords

In [364]:
def create_coded(X):
    return np.zeros(len(X))


In [365]:
def encode(X, coded_X):
    for j, k in enumerate(X):
        for i, unique_chord in enumerate(unique_chords):
            if np.array_equal(k, unique_chord):
                coded_X[j] = i

In [ ]:
model_chords = Sequential()
model_notes = Sequential()

model_chords.add(Embedding(38113, 20, input_length=1, batch_input_shape=(1,1)))
model_notes.add(Embedding(128, 20, input_length=1, batch_input_shape=(1,1)))

merged_model = Sequential()
merged = Merge([model_chords, model_notes])
merged_model.add(merged)
merged_model.add(LSTM(20, stateful=True))
merged_model.add(Dense(38113, activation='softmax'))
merged_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

y_real = []
y_pred = []
unique_chords = np.zeros(4) - 1

with open('gtp_dataset_final.pickle', 'rb') as input_file:
    i = 0
    while True:
        try:
            i += 1
            song = ml.structures.Song()
            song.undump(input_file)
            chords = []
            notes = []
            flag_notes_chords = cut_song(song, notes, chords)
            if (flag_notes_chords[0] == 0) :
                continue
            notes = flag_notes_chords[1]
            chords = flag_notes_chords[2]
            X = []
            Y = []
            X, Y = create_X_Y(chords)
            X = norm_array(X, 4)
            Y = norm_array(Y, 4)
            notes_to_np(notes)
            unique_chords = create_unique_chords(X, unique_chords)
            unique_chords = create_unique_chords(Y, unique_chords)
            unique_chords = unique_chords.reshape((-1, 4))
            unique_chords = np.unique(unique_chords, axis=0)
            unique_chords = list(unique_chords)
            coded_X = []
            coded_Y = []
            coded_X = create_coded(X)
            coded_Y = create_coded(Y)
            encode(X, coded_X)
            encode(Y, coded_Y)
            X = np.array(coded_X)
            y = np.array(coded_Y)
            X_notes = notes
            X_notes = X_notes[:len(X)]
            X = X[:len(X_notes)]
            y = y[:len(X)]
            if (i < 27100):
                merged_model.fit([X, X_notes], y, epochs=1, batch_size=1, verbose=1)
                merged_model.reset_states()
            else:
                y_pred.append(merged_model.predict([X, X_notes], batch_size=1, verbose=1).argmax(axis=1))
                y_real.append(y)
            
            
        except EOFError:
            break

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  


Epoch 1/1
446/446 [==============================] - 23s 52ms/step - loss: 5.2626 - acc: 0.1570
Epoch 1/1
286/286 [==============================] - 12s 42ms/step - loss: 2.5484 - acc: 0.1818
Epoch 1/1
90/90 [==============================] - 4s 43ms/step - loss: 2.9121 - acc: 0.2222
Epoch 1/1
90/90 [==============================] - 4s 43ms/step - loss: 2.4810 - acc: 0.3111
Epoch 1/1
94/94 [==============================] - 4s 42ms/step - loss: 10.8608 - acc: 0.0000e+00
Epoch 1/1
94/94 [==============================] - 4s 46ms/step - loss: 7.9213 - acc: 0.0213
Epoch 1/1
114/114 [==============================] - 6s 52ms/step - loss: 9.1920 - acc: 0.0000e+00
Epoch 1/1
94/94 [==============================] - 4s 40ms/step - loss: 6.9091 - acc: 0.0000e+00
Epoch 1/1
94/94 [==============================] - 4s 38ms/step - loss: 4.8958 - acc: 0.0000e+00
Epoch 1/1
644/644 [==============================] - 27s 42ms/step - loss: 5.7832 - acc: 0.0435
Epoch 1/1
516/516 [=======================

1806/1806 [==============================] - 70s 39ms/step - loss: 3.8656 - acc: 0.4236
Epoch 1/1
62/62 [==============================] - 2s 39ms/step - loss: 8.8197 - acc: 0.0000e+00
Epoch 1/1
62/62 [==============================] - 2s 40ms/step - loss: 3.1948 - acc: 0.2581
Epoch 1/1
38/38 [==============================] - 1s 39ms/step - loss: 6.7514 - acc: 0.0000e+00
Epoch 1/1
94/94 [==============================] - 4s 39ms/step - loss: 2.9326 - acc: 0.2553
Epoch 1/1
62/62 [==============================] - 2s 40ms/step - loss: 1.2900 - acc: 0.7419
Epoch 1/1
94/94 [==============================] - 4s 40ms/step - loss: 1.6034 - acc: 0.7553
Epoch 1/1
62/62 [==============================] - 2s 39ms/step - loss: 0.9823 - acc: 0.8710
Epoch 1/1
62/62 [==============================] - 2s 39ms/step - loss: 0.7468 - acc: 0.8226
Epoch 1/1
94/94 [==============================] - 4s 39ms/step - loss: 2.4066 - acc: 0.5745
Epoch 1/1
38/38 [==============================] - 1s 39ms/step - l

202/202 [==============================] - 8s 39ms/step - loss: 7.9148 - acc: 0.1634
Epoch 1/1
252/252 [==============================] - 10s 38ms/step - loss: 5.1865 - acc: 0.3413
Epoch 1/1
378/378 [==============================] - 15s 39ms/step - loss: 2.9788 - acc: 0.4418
Epoch 1/1
106/106 [==============================] - 4s 38ms/step - loss: 13.4162 - acc: 0.1226
Epoch 1/1
254/254 [==============================] - 10s 38ms/step - loss: 11.4801 - acc: 0.1732
Epoch 1/1
286/286 [==============================] - 11s 38ms/step - loss: 9.2257 - acc: 0.0524
Epoch 1/1
126/126 [==============================] - 5s 40ms/step - loss: 5.0840 - acc: 0.2460
Epoch 1/1
126/126 [==============================] - 5s 39ms/step - loss: 1.6674 - acc: 0.7698
Epoch 1/1
142/142 [==============================] - 5s 39ms/step - loss: 1.0545 - acc: 0.8028
Epoch 1/1
158/158 [==============================] - 6s 39ms/step - loss: 9.8957 - acc: 0.2911
Epoch 1/1
1278/1278 [==============================] -

2248/2248 [==============================] - 86s 38ms/step - loss: 0.7799 - acc: 0.8986
Epoch 1/1
93/93 [==============================] - 4s 38ms/step - loss: 9.4915 - acc: 0.2258
Epoch 1/1
1550/1550 [==============================] - 59s 38ms/step - loss: 3.6572 - acc: 0.2813
Epoch 1/1
254/254 [==============================] - 10s 38ms/step - loss: 9.6830 - acc: 0.2244
Epoch 1/1
104/104 [==============================] - 4s 38ms/step - loss: 4.4757 - acc: 0.5000
Epoch 1/1
110/110 [==============================] - 4s 38ms/step - loss: 3.1164 - acc: 0.4636
Epoch 1/1
110/110 [==============================] - 4s 38ms/step - loss: 2.3001 - acc: 0.5182
Epoch 1/1
40/40 [==============================] - 2s 39ms/step - loss: 9.9585 - acc: 0.0000e+00
Epoch 1/1
142/142 [==============================] - 5s 38ms/step - loss: 11.1081 - acc: 0.0986
Epoch 1/1
58/58 [==============================] - 2s 38ms/step - loss: 6.4211 - acc: 0.4655
Epoch 1/1
58/58 [==============================] - 2s 

...

  41/1746 [..............................] - ETA: 6:37:42 - loss: 14.9985 - acc: 0.0244

/usr/local/lib/python3.5/dist-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.182575). Check your callbacks.
  % delta_t_median)


  44/1746 [..............................] - ETA: 6:11:02 - loss: 15.0137 - acc: 0.0227

/usr/local/lib/python3.5/dist-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.109878). Check your callbacks.
  % delta_t_median)


1746/1746 [==============================] - 718s 412ms/step - loss: 8.2828 - acc: 0.0928
Epoch 1/1
734/734 [==============================] - 40s 54ms/step - loss: 10.8996 - acc: 0.0341
Epoch 1/1
268/268 [==============================] - 18s 68ms/step - loss: 8.4046 - acc: 0.1157
Epoch 1/1
382/382 [==============================] - 19s 50ms/step - loss: 9.0084 - acc: 0.1099
Epoch 1/1
442/442 [==============================] - 59s 134ms/step - loss: 7.1380 - acc: 0.2172
Epoch 1/1
62/62 [==============================] - 3s 51ms/step - loss: 11.1719 - acc: 0.1774
Epoch 1/1
126/126 [==============================] - 9s 71ms/step - loss: 13.0976 - acc: 0.0000e+00
Epoch 1/1
78/78 [==============================] - 5s 64ms/step - loss: 9.8005 - acc: 0.0000e+00
Epoch 1/1
62/62 [==============================] - 4s 61ms/step - loss: 13.0119 - acc: 0.0000e+00
Epoch 1/1
126/126 [==============================] - 8s 61ms/step - loss: 6.9757 - acc: 0.0000e+00
Epoch 1/1
78/78 [===================

In [369]:
import pandas as pd

df = pd.DataFrame(unique_chords)
df.to_csv("vocab.csv")
x = pd.read_csv("vocab.csv")
x = x.values.T[1:].T


In [370]:
merged_model.save("NN_model.h5")

In [374]:
model_chords = Sequential()
model_notes = Sequential()

model_chords.add(Embedding(38113, 20, input_length=1, batch_input_shape=(1,1)))
model_notes.add(Embedding(128, 20, input_length=1, batch_input_shape=(1,1)))

model = Sequential()
merged = Merge([model_chords, model_notes])
model.add(merged)
model.add(LSTM(20, stateful=True))
model.add(Dense(38113, activation='softmax'))

model.load_weights("NN_model.h5")
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  


In [375]:
unique_chords[model.predict([np.array([40]), np.array([22.])], batch_size=1, verbose=1).argmax()]

1/1 [==============================] - 5s 5s/step


array([33., 40., 42., 73.])